### Importing Libraries


In [ ]:
import pandas as pd
import numpy as np
import keras 
import tensorflow as tf

### Checking for GPU

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


### Reading the CSV file

In [ ]:
train=pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
sample=pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")


### Adding path of the image in csv file

In [ ]:
directory="../input/petfinder-pawpularity-score/train"

path=[]

for i in train['Id']:
    path.append(f'{directory}/{i}.jpg')
train['path'] = path     

X=train[["path","Pawpularity"]]

### ImageDataGenerator from keras

In [ ]:
gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2,
    #brightness_range = (0.1,1.0),
)

### forming train and validation dataset

In [ ]:
train_images = gen.flow_from_dataframe(
    dataframe = X,
    x_col = 'path',
    y_col = 'Pawpularity',
    target_size =(180,180),
    color_mode='rgb',
    class_mode = "raw",
    batch_size = 64,
    shuffle= True,
    seed=42,
    subset= 'training',
)


validation_images = gen.flow_from_dataframe(
    dataframe = X,
    x_col = 'path',
    y_col = 'Pawpularity',
    target_size =(180,180),
    color_mode='rgb',
    class_mode = "raw",
    batch_size=64,
    shuffle= True,
    seed=42,
    subset= 'validation',
)

### Model

In [ ]:
model = keras.Sequential(
    [
     keras.layers.Conv2D(filters=16,padding='same',kernel_size=(3,3),activation='relu',input_shape=(180,180,3)),
     keras.layers.BatchNormalization(),
     keras.layers.MaxPool2D(strides=(2,2)),
     keras.layers.Conv2D(filters=32,padding='same',kernel_size=(3,3),activation='relu'),
     keras.layers.MaxPool2D(strides=(2,2)),
     keras.layers.Conv2D(filters=32,padding='same',kernel_size=(3,3),activation='relu'),
     keras.layers.MaxPool2D(strides=(2,2)),
     keras.layers.GlobalAveragePooling2D(),
     keras.layers.Dense(512,activation = 'relu'),
     keras.layers.Dense(1024,activation = 'relu'),
     keras.layers.Dense(1,activation='linear')
    ])

### Training the model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(
                    train_images,
                    validation_data = validation_images,
                    epochs=20,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)])

In [ ]:
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
    #brightness_range = (0.1,1.0),
)

import os
test_files_df=pd.DataFrame()
test_files_df['file']=os.listdir('../input/petfinder-pawpularity-score/test')

test_images = test_gen.flow_from_dataframe(
    dataframe = test_files_df,
    directory="../input/petfinder-pawpularity-score/test",
    x_col = 'file',
    y_col = None,
    #has_ext=True,
    target_size =(180,180),
    color_mode='rgb',
    class_mode = None,
    batch_size = 64,
    shuffle= True,
    seed=42,
)

In [ ]:
pred=model.predict(test_images)
sample.Pawpularity=pred
sample.to_csv("submission.csv",index=False)